# 4.9.1 Integrating Customer Data Set 

## Wrangling, Cleaning, Combining with orders/products data

## Contents 

1. Import libraries, ords_prods_merge and customer.csv data
2. Data Wrangling customer data
    - dropping first_name, surnam columns
    - changing data types
3. Data Quality and consistency checks
    - duplicate values check
    - mixed values check
    - missing values check
4. Merging customer data with ords_prods data
    - checking for common variable for merge point
    - overcoming errors via stackoverflow guidance
    - inspecting combined df ords_prods_all
5. Exporting data via pickle format
    
    

### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# Setting path to Achievement 4 master folder
path = r'/Users/jeremyobach/Documents/Data Analytics/CareerFoundry/Achievement 4- Python for DA/DEC23 Instacart Basket Analysis - MASTER FOLDER'
#checking path set correctly
path

'/Users/jeremyobach/Documents/Data Analytics/CareerFoundry/Achievement 4- Python for DA/DEC23 Instacart Basket Analysis - MASTER FOLDER'

### Importing customers.csv as customers df
### Importing ords_prods_merge_48 (latest version from ex. 4.8) as ords_prods_merge  df

In [3]:
customers = pd.read_csv(os.path.join(path,'02 Data', 'Original Data', 'customers.csv'), index_col=False)

In [4]:
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data','ords_prods_merge_48.pkl'))

In [5]:
customers.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [6]:
customers.shape

(206209, 10)

Initial thoughts from customers df: 
- user id will probably be used to link to ords_prods_merge df
- might want to drop first/last name columns
- unsure about date_joined utility
- everything else could be useful info to cross-examine the dataset.

### Data Wrangling

In [7]:
# Dropping first name/ surname columns, saving to "df"

df = customers.drop(columns = ['First Name', 'Surnam'])

In [8]:
df.tail()

,user_id,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
206204,168073,Female,North Carolina,44,4/1/2020,1,married,148828
206205,49635,Male,Hawaii,62,4/1/2020,3,married,168639
206206,135902,Female,Missouri,66,4/1/2020,2,married,53374
206207,81095,Female,California,27,4/1/2020,1,married,99799
206208,80148,Female,New York,55,4/1/2020,1,married,57095


In [9]:
df.describe()

,user_id,Age,n_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


In [10]:
# Changing user_id to string
df['user_id'] = df['user_id'].astype('str')

In [11]:
df['user_id'].dtype

dtype('O')

In [12]:
df.describe()

,Age,n_dependants,income
count,206209.000000,206209.000000,206209.000000
mean,49.501646,1.499823,94632.852548
std,18.480962,1.118433,42473.786988
min,18.000000,0.000000,25903.000000
25%,33.000000,0.000000,59874.000000
50%,49.000000,1.000000,93547.000000
75%,66.000000,3.000000,124244.000000
max,81.000000,3.000000,593901.000000


### Data Quality and Consistency Checks

In [13]:
# Mixed data type check

for col in df.columns.tolist():
  weird = (df[[col]].map(type) != df[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df[weird]) > 0:
    print (col)

No output, so no mixed data types

In [14]:
# Checking for duplicates

df_dups = df[df.duplicated]

In [15]:
df_dups.shape

(0, 8)

In [16]:
df_dups.head(8)

,user_id,Gender,STATE,Age,date_joined,n_dependants,fam_status,income


No duplicates either

In [17]:
# Checking for missing values

df.isnull().sum()

user_id         0
Gender          0
STATE           0
Age             0
date_joined     0
n_dependants    0
fam_status      0
income          0
dtype: int64

In [18]:
df.shape

(206209, 8)

No missing values, check!

### Combining customers df with ords_prods_merge df
First let's refresh on what ords_prods_merge looks like atp

In [19]:
ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_week,orders_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,...,prices,price_range_loc,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_prod_price,spending_type,ordering_behavior,frequency_flag
0,2539329,1,1,2,8,NaN,True,196,1,0,...,9.0,Mid range product,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent_customer
1,2398795,1,2,3,7,15.0,False,196,1,1,...,9.0,Mid range product,Slowest days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent_customer
2,473747,1,3,3,12,21.0,False,196,1,1,...,9.0,Mid range product,Slowest days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent_customer
3,2254736,1,4,4,7,29.0,False,196,1,1,...,9.0,Mid range product,Slowest days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent_customer
4,431534,1,5,4,15,28.0,False,196,1,1,...,9.0,Mid range product,Slowest days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent_customer


In [20]:
ords_prods_merge.shape

(32404859, 23)

In [21]:
ords_prods_merge.columns

Index(['order_id', 'user_id', 'order_number', 'orders_day_of_week',
       'orders_hour_of_day', 'days_since_prior_order', 'first_order',
       'product_id', 'add_to_cart_order', 'reordered', 'product_name',
       'aisle_id', 'department_id', 'prices', 'price_range_loc',
       'busiest_days', 'busiest_period_of_day', 'max_order', 'loyalty_flag',
       'avg_prod_price', 'spending_type', 'ordering_behavior',
       'frequency_flag'],
      dtype='object')

'user_id' definitely the merge point. A "left merge" to include all the orders and prods and have the customer info fill in makes sense in this case.

In [22]:
# Ensuring both df's have 'user_id' datatype as 'str'
#ords_prods_merge = ords_prods_merge['user_id'].astype('str')

In [23]:
df['user_id'].dtype

dtype('O')

In [24]:
df.head()

,user_id,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Female,Maryland,26,1/1/2017,1,married,40374


### Merge errors kept occuring, saw on stackoverflow how to amend this error:
'ValueError: You are trying to merge on object and int64 columns when use pandas merge'
Basically, to cast the user_id as a str in both dataframes to be merged. Thought I had verified both of them as strings before but this method worked

In [25]:
ords_prods_merge['user_id'] = ords_prods_merge['user_id'].astype(str)

In [26]:
df['user_id'] = df['user_id'].astype(str)

In [27]:
ords_prods_all = ords_prods_merge.merge(df, on = ['user_id'], indicator = True, how = 'left')
#ords_prods_all = ords_prods_merge.merge(df, on = ['user_id'], indicator = True)

In [28]:
# We got a merge!!

ords_prods_all.head(25)

,order_id,user_id,order_number,orders_day_of_week,orders_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,...,ordering_behavior,frequency_flag,Gender,STATE,Age,date_joined,n_dependants,fam_status,income,_merge
0,2539329,1,1,2,8,NaN,True,196,1,0,...,20.5,Non-frequent_customer,Female,Alabama,31,2/17/2019,3,married,40423,both
1,2398795,1,2,3,7,15.0,False,196,1,1,...,20.5,Non-frequent_customer,Female,Alabama,31,2/17/2019,3,married,40423,both
2,473747,1,3,3,12,21.0,False,196,1,1,...,20.5,Non-frequent_customer,Female,Alabama,31,2/17/2019,3,married,40423,both
3,2254736,1,4,4,7,29.0,False,196,1,1,...,20.5,Non-frequent_customer,Female,Alabama,31,2/17/2019,3,married,40423,both
4,431534,1,5,4,15,28.0,False,196,1,1,...,20.5,Non-frequent_customer,Female,Alabama,31,2/17/2019,3,married,40423,both
5,3367565,1,6,2,7,19.0,False,196,1,1,...,20.5,Non-frequent_customer,Female,Alabama,31,2/17/2019,3,married,40423,both
6,550135,1,7,1,9,20.0,False,196,1,1,...,20.5,Non-frequent_customer,Female,Alabama,31,2/17/2019,3,married,40423,both
7,3108588,1,8,1,14,14.0,False,196,2,1,...,20.5,Non-frequent_customer,Female,Alabama,31,2/17/2019,3,married,40423,both
8,2295261,1,9,1,16,0.0,False,196,4,1,...,20.5,Non-frequent_customer,Female,Alabama,31,2/17/2019,3,married,40423,both
9,2550362,1,10,4,8,30.0,False,196,1,1,...,20.5,Non-frequent_customer,Female,Alabama,31,2/17/2019,3,married,40423,both


In [29]:
ords_prods_all.shape

# Same number of rows as pre-merge, with added customer columns

(32404859, 31)

In [30]:
ords_prods_all['_merge'].value_counts()

# Looks like a full match, despite "left" merge type. Could be the nature of the data, but something to note.

_merge
both          32404859
left_only            0
right_only           0
Name: count, dtype: int64

## To recap, customer dataset was:
- imported and explored,
- wrangled (left out first_name and surname), 
- checked for mixed data, duplicates, and missing values
- merged with latest version of ords_prods on 'user_id'. 

In [31]:
ords_prods_all.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_all.pkl'))